# load karaman-davis dataset (from kissim_app)

In [1]:
import sys
import os

# get the root path of the project adn add it to the system path
# This is necessary to import the modules from the project
root_path = os.path.abspath(os.path.join(os.getcwd(), "../../../kissim_app"))
print(root_path)
sys.path.append(root_path)


/Users/yanyz/Projects/kissim_mdfp/kissim_app


In [2]:
from src.data.kinases import kinmap
from src.data.profiling import load


karaman_davis_reduced = load('karaman-davis', pkidb_ligands=True, fda_approved=False)

Folder does not exist: /Users/yanyz/Projects/kissim_mdfp/kissim_app/src/../results
Use test folder instead: /Users/yanyz/Projects/kissim_mdfp/kissim_app/src/../test/results
Folder does not exist: /Users/yanyz/Projects/kissim_mdfp/kissim_app/src/../data/external/structures/20210902_KLIFS_HUMAN
Use test folder instead: /Users/yanyz/Projects/kissim_mdfp/kissim_app/src/../test/data/external/klifs_test


Changed ligand names (unknown names may be discarded - see function docstring):
           ligand.input            ligand.pkidb
0         Staurosporine  unknown (not in PKIDB)
1           JNJ-7706621  unknown (not in PKIDB)
2              MLN-8054  unknown (not in PKIDB)
3               PKC-412             Midostaurin
4              SU-14813  unknown (not in PKIDB)
5             SB-202190  unknown (not in PKIDB)
6             CP-724714  unknown (not in PKIDB)
7        VX-680/MK-0457  unknown (not in PKIDB)
9   Roscovitine/CYC-202  unknown (not in PKIDB)
10            SB-203580  unknown (not in PKIDB)
12            CP-690550             Tofacitinib
14              GW-2580  unknown (not in PKIDB)
16            GW-786034  unknown (not in PKIDB)
18               VX-745            Neflamapimod
19            SB-431542  unknown (not in PKIDB)
20              ZD-6474              Vandetanib
22         Flavopiridol               Alvocidib
24              MLN-518              Tandutinib
25      

In [3]:
kinase_names = ['ABL1', 'DDR1', 'EphA2', 'LOK', 'MYT1']

karaman_davis_reduced_subset = karaman_davis_reduced[karaman_davis_reduced.index.isin(kinase_names)]
df_karaman_davis = karaman_davis_reduced_subset[['Dasatinib', 'Bosutinib']]
df_karaman_davis

,Dasatinib,Bosutinib
ABL1,0.046,0.057
DDR1,0.690,120.000
EphA2,0.850,18.000
LOK,1200.000,7.000
MYT1,130.000,350.000


* LOK, ABL1; Bosutinib, Dasatinib
* LOK, EphA2; Bosutinib, Dasatinib
* LOK, MYT1; Bosutinib, Dasatinib
* ABL1, DDR1; Dasatinib, Imatinib
* ABL1, EphA2; Bosutinib, Dasatinib
* ABL1, MYT1; Bosutinib, Dasatinib
* EphA2, MYT1; Bosutinib, Dasatinib

# Fetch structures from klifs

In [4]:
from opencadd.databases.klifs import setup_remote

klifs_session = setup_remote()

In [5]:
# Get all available structures for these kinases
structures_df = klifs_session.structures.by_kinase_name(kinase_names=kinase_names)
print(f"Number of structures: {len(structures_df)}")
print("Kinases:", *structures_df["kinase.klifs_name"].unique())

Number of structures: 494
Kinases: MYT1 LOK ABL1 DDR1 EphA2


In [6]:
structures_df.columns

Index(['structure.klifs_id', 'structure.pdb_id', 'structure.alternate_model',
       'structure.chain', 'species.klifs', 'kinase.klifs_id',
       'kinase.klifs_name', 'kinase.names', 'kinase.family', 'kinase.group',
       'structure.pocket', 'ligand.expo_id', 'ligand_allosteric.expo_id',
       'ligand.klifs_id', 'ligand_allosteric.klifs_id', 'ligand.name',
       'ligand_allosteric.name', 'structure.dfg', 'structure.ac_helix',
       'structure.resolution', 'structure.qualityscore',
       'structure.missing_residues', 'structure.missing_atoms',
       'structure.rmsd1', 'structure.rmsd2', 'interaction.fingerprint',
       'structure.front', 'structure.gate', 'structure.back', 'structure.fp_i',
       'structure.fp_ii', 'structure.bp_i_a', 'structure.bp_i_b',
       'structure.bp_ii_in', 'structure.bp_ii_a_in', 'structure.bp_ii_b_in',
       'structure.bp_ii_out', 'structure.bp_ii_b', 'structure.bp_iii',
       'structure.bp_iv', 'structure.bp_v', 'structure.grich_distance',
       

## Filter structures

We filter the structures by different criteria:

- Species: human
- Conformation: DFG-in (the active kinase conformation)
- Resolution: <= 3 Angström
- Quality score*: >= 6

\* The KLIFS quality score takes into account the quality of the alignment, as well as the number of missing residues and atoms. A higher score indicates a better structure quality.

In [7]:
'''
structures_df = structures_df[
    (structures_df["species.klifs"] == "Human")
    & (structures_df["structure.dfg"] == "in")
    & (structures_df["structure.resolution"] <= 3)
    & (structures_df["structure.qualityscore"] >= 6)
]
print(f"Number of structures: {len(structures_df)}")
print("Kinases:", *structures_df["kinase.klifs_name"].unique())
print("Count the number of structures per kinase:", structures_df["kinase.klifs_name"].value_counts())
structures_df
'''

'\nstructures_df = structures_df[\n    (structures_df["species.klifs"] == "Human")\n    & (structures_df["structure.dfg"] == "in")\n    & (structures_df["structure.resolution"] <= 3)\n    & (structures_df["structure.qualityscore"] >= 6)\n]\nprint(f"Number of structures: {len(structures_df)}")\nprint("Kinases:", *structures_df["kinase.klifs_name"].unique())\nprint("Count the number of structures per kinase:", structures_df["kinase.klifs_name"].value_counts())\nstructures_df\n'

- 'Dasatinib' = 1N1
- 'Bosutinib' = DB8

In [8]:
# get all the data containing "ligand" in column name
df_dasatinib_bosutinib = structures_df[
    (structures_df['ligand.expo_id'] == '1N1') | (structures_df['ligand.expo_id'] == 'DB8')
]
df_dasatinib_bosutinib[['structure.klifs_id', 'structure.pdb_id', 'kinase.klifs_name', 'species.klifs', 'structure.dfg', 'structure.resolution', 'structure.qualityscore', 'ligand.expo_id']]

,structure.klifs_id,structure.pdb_id,kinase.klifs_name,species.klifs,structure.dfg,structure.resolution,structure.qualityscore,ligand.expo_id
11,8339,5vcv,MYT1,Human,in,1.92,8.0,1N1
17,8345,5vcy,MYT1,Human,in,1.56,8.0,DB8
18,8338,5vcy,MYT1,Human,in,1.56,8.0,DB8
24,948,5ajq,LOK,Human,in,2.20,7.6,DB8
25,940,5ajq,LOK,Human,in,2.20,7.6,DB8
26,955,5ajq,LOK,Human,in,2.20,9.6,DB8
52,950,5ajq,LOK,Human,in,2.20,9.6,DB8
55,8384,5owr,LOK,Human,in,2.30,6.4,1N1
56,8388,5owr,LOK,Human,in,2.30,6.4,1N1
92,1094,4xey,ABL1,Human,out-like,2.89,6.4,1N1


In [9]:
# for the same pdb, select the top 1 structure per kinase in terms of KLIFS quality score
df_dasatinib_bosutinib = df_dasatinib_bosutinib.sort_values('structure.qualityscore', ascending=False)
df_dasatinib_bosutinib = df_dasatinib_bosutinib.drop_duplicates(subset=['structure.pdb_id', 'kinase.klifs_name'])
df_dasatinib_bosutinib[['structure.klifs_id', 'structure.pdb_id', 'kinase.klifs_name', 'species.klifs', 'structure.dfg', 'structure.resolution', 'structure.qualityscore', 'ligand.expo_id']]

,structure.klifs_id,structure.pdb_id,kinase.klifs_name,species.klifs,structure.dfg,structure.resolution,structure.qualityscore,ligand.expo_id
26,955,5ajq,LOK,Human,in,2.20,9.6,DB8
147,1057,3ue4,ABL1,Human,out-like,2.42,8.9,DB8
266,10761,6bsd,DDR1,Human,out-like,2.61,8.4,1N1
11,8339,5vcv,MYT1,Human,in,1.92,8.0,1N1
17,8345,5vcy,MYT1,Human,in,1.56,8.0,DB8
401,7177,5i9x,EphA2,Human,in,1.43,8.0,DB8
392,7176,5i9y,EphA2,Human,in,1.23,8.0,1N1
219,1053,2gqg,ABL1,Human,in,2.40,8.0,1N1
103,14197,7n9g,ABL1,Human,out-like,2.20,8.0,1N1
92,1094,4xey,ABL1,Human,out-like,2.89,6.4,1N1


In [10]:
df_dasatinib_bosutinib[['kinase.klifs_name', 'ligand.expo_id']]

,kinase.klifs_name,ligand.expo_id
26,LOK,DB8
147,ABL1,DB8
266,DDR1,1N1
11,MYT1,1N1
17,MYT1,DB8
401,EphA2,DB8
392,EphA2,1N1
219,ABL1,1N1
103,ABL1,1N1
92,ABL1,1N1


## Merge Karaman-Davis bioactivity data into kissim df

In [11]:
df_karaman_davis_long = df_karaman_davis.rename(columns={'Dasatinib': '1N1', 'Bosutinib': 'DB8'})

df_karaman_davis_long = df_karaman_davis_long.reset_index().melt(id_vars='index', var_name='ligand.expo_id', value_name='kinmap.bioactivity')
df_karaman_davis_long = df_karaman_davis_long.rename(columns={'index': 'kinase.klifs_name'})

df_karaman_davis_long

,kinase.klifs_name,ligand.expo_id,kinmap.bioactivity
0,ABL1,1N1,0.046
1,DDR1,1N1,0.690
2,EphA2,1N1,0.850
3,LOK,1N1,1200.000
4,MYT1,1N1,130.000
5,ABL1,DB8,0.057
6,DDR1,DB8,120.000
7,EphA2,DB8,18.000
8,LOK,DB8,7.000
9,MYT1,DB8,350.000


In [12]:
df_merged = df_dasatinib_bosutinib.merge(df_karaman_davis_long, on=['kinase.klifs_name', 'ligand.expo_id'], how='left')
df_merged[['structure.klifs_id', 'structure.pdb_id', 'kinase.klifs_name', 'species.klifs', 'structure.dfg', 'structure.resolution', 'structure.qualityscore', 'ligand.expo_id', 'kinmap.bioactivity']]

,structure.klifs_id,structure.pdb_id,kinase.klifs_name,species.klifs,structure.dfg,structure.resolution,structure.qualityscore,ligand.expo_id,kinmap.bioactivity
0,955,5ajq,LOK,Human,in,2.20,9.6,DB8,7.000
1,1057,3ue4,ABL1,Human,out-like,2.42,8.9,DB8,0.057
2,10761,6bsd,DDR1,Human,out-like,2.61,8.4,1N1,0.690
3,8339,5vcv,MYT1,Human,in,1.92,8.0,1N1,130.000
4,8345,5vcy,MYT1,Human,in,1.56,8.0,DB8,350.000
5,7177,5i9x,EphA2,Human,in,1.43,8.0,DB8,18.000
6,7176,5i9y,EphA2,Human,in,1.23,8.0,1N1,0.850
7,1053,2gqg,ABL1,Human,in,2.40,8.0,1N1,0.046
8,14197,7n9g,ABL1,Human,out-like,2.20,8.0,1N1,0.046
9,1094,4xey,ABL1,Human,out-like,2.89,6.4,1N1,0.046


# Calculate KiSSim fingerprints

In [13]:
from kissim.api import encode
import numpy as np
import pandas as pd

In [14]:
structure_klifs_ids = df_dasatinib_bosutinib['structure.klifs_id']

kissim_fingerprints = encode(structure_klifs_ids)

In [15]:
kissim_fingerprints_array = [
    fingerprint.values_array().tolist()
    for _, fingerprint in kissim_fingerprints.data.items()
]
kissim_fingerprints_array = np.array(kissim_fingerprints_array)
kissim_fingerprints_df = pd.DataFrame(kissim_fingerprints_array, index=structure_klifs_ids)

print(f"Matrix shape: {kissim_fingerprints_df.shape}")
print(f"Number of fingerprints: {kissim_fingerprints_df.shape[0]}")
print(f"Number of fingerprint bits: {kissim_fingerprints_df.shape[1]}")
kissim_fingerprints_df

Matrix shape: (12, 1032)
Number of fingerprints: 12
Number of fingerprint bits: 1032


,0,1,2,3,4,5,6,7,8,9,...,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031
structure.klifs_id,,,,,,,,,,,,,,,,,,,,,
955,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,2.0,0.0,...,12.935410,11.943754,4.392296,4.712463,4.455708,3.494710,2.748480,2.987596,3.679778,1.538272
1057,2.0,1.0,1.0,0.0,1.0,0.0,3.0,3.0,2.0,1.0,...,13.025692,12.011242,4.460881,5.034019,4.077518,3.423783,2.929926,3.687617,3.166885,2.050392
10761,2.0,0.0,2.0,-1.0,0.0,0.0,NaN,3.0,2.0,1.0,...,12.857004,12.029064,4.471011,5.032515,4.133732,3.345548,3.054943,3.715677,2.721924,1.939075
8339,1.0,1.0,1.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,...,12.970246,11.975555,4.590457,4.656574,4.256000,3.443614,2.799831,3.435526,3.402649,1.802981
8345,1.0,1.0,1.0,0.0,0.0,0.0,2.0,3.0,3.0,3.0,...,12.979882,11.973692,4.545204,4.679642,4.239791,3.412097,2.645016,3.439387,3.357879,1.488037
7177,2.0,1.0,0.0,1.0,0.0,0.0,3.0,3.0,1.0,0.0,...,13.156184,12.171861,4.448386,4.729081,4.016259,3.348171,2.459051,3.189591,2.456419,1.516050
7176,2.0,1.0,0.0,1.0,0.0,0.0,3.0,3.0,1.0,0.0,...,13.144528,12.173165,4.458933,4.673591,4.044846,3.318169,2.554392,3.143976,2.470045,1.537785
1053,2.0,1.0,1.0,0.0,1.0,0.0,3.0,3.0,2.0,1.0,...,12.822959,11.962476,4.493169,4.836616,4.315794,3.468795,3.217910,3.302662,3.380991,2.067249
14197,2.0,1.0,1.0,0.0,1.0,0.0,2.0,3.0,2.0,1.0,...,12.813321,11.984674,4.472458,5.047574,4.216868,3.430281,2.944593,3.567419,2.978940,1.621562


In [88]:
for key, fingerprint in kissim_fingerprints.data.items():
    print(key, len(fingerprint.values_array()))

955 1032
1057 1032
10761 1032
8339 1032
8345 1032
7177 1032
7176 1032
1053 1032
14197 1032
1094 1032
8388 1032
3254 1032


In [25]:
# save the fingerprints to a csv file
kissim_fingerprints_df.to_csv('data/kissim_fingerprints.csv', index=True)

# Compare structures

In [16]:
from sklearn.metrics.pairwise import nan_euclidean_distances

In [17]:
structure_distance_matrix_array = nan_euclidean_distances(kissim_fingerprints_df.values)

# Create DataFrame with structure KLIFS IDs as index/columns
structure_klifs_ids = kissim_fingerprints_df.index.to_list()
structure_distance_matrix_df = pd.DataFrame(
    structure_distance_matrix_array, index=structure_klifs_ids, columns=structure_klifs_ids
)
print(f"Structure distance matrix size: {structure_distance_matrix_df.shape}")
structure_distance_matrix_df

Structure distance matrix size: (12, 12)


,955,1057,10761,8339,8345,7177,7176,1053,14197,1094,8388,3254
955,0.000000,28.576142,29.268622,27.070272,25.880838,32.049004,32.338314,27.345969,26.987305,27.060062,20.127414,29.535628
1057,28.576142,0.000000,20.448719,24.263294,24.275118,22.898073,23.211865,17.856452,17.480245,8.191927,20.824617,20.386950
10761,29.268622,20.448719,0.000000,24.935382,26.078996,23.450612,23.192182,26.258210,27.292591,17.887989,23.689144,7.628437
8339,27.070272,24.263294,24.935382,0.000000,6.680548,25.098772,25.161640,25.959846,29.273259,23.767301,23.136211,24.507839
8345,25.880838,24.275118,26.078996,6.680548,0.000000,25.598288,25.751657,26.249652,28.653591,23.771006,23.395898,25.658868
7177,32.049004,22.898073,23.450612,25.098772,25.598288,0.000000,3.898270,27.826531,29.753711,19.909221,21.697771,22.961274
7176,32.338314,23.211865,23.192182,25.161640,25.751657,3.898270,0.000000,27.827668,29.961407,20.418159,22.328941,22.773133
1053,27.345969,17.856452,26.258210,25.959846,26.249652,27.826531,27.827668,0.000000,18.140601,16.351846,20.976468,25.701912
14197,26.987305,17.480245,27.292591,29.273259,28.653591,29.753711,29.961407,18.140601,0.000000,12.611501,22.060313,27.975786
1094,27.060062,8.191927,17.887989,23.767301,23.771006,19.909221,20.418159,16.351846,12.611501,0.000000,20.853383,18.341074


In [22]:
subset_labels = [8388, 1057, 1053, 955]
structure_distance_matrix_df.loc[subset_labels, subset_labels]

,8388,1057,1053,955
8388,0.000000,20.824617,20.976468,20.127414
1057,20.824617,0.000000,17.856452,28.576142
1053,20.976468,17.856452,0.000000,27.345969
955,20.127414,28.576142,27.345969,0.000000


# Map structure to kinase distance matrix¶

In [18]:
# Copy distance matrix to kinase matrix
kinase_distance_matrix_df = structure_distance_matrix_df.copy()
# Replace structure KLIFS IDs with the structures' kinase names
kinase_names = structures_df.set_index("structure.klifs_id").loc[
    structure_klifs_ids, "kinase.klifs_name"
]
kinase_distance_matrix_df.index = kinase_names
kinase_distance_matrix_df.columns = kinase_names
print("Show matrix subset:")
kinase_distance_matrix_df
# NBVAL_CHECK_OUTPUT


Show matrix subset:


kinase.klifs_name,LOK,ABL1,DDR1,MYT1,MYT1,EphA2,EphA2,ABL1,ABL1,ABL1,LOK,DDR1
kinase.klifs_name,,,,,,,,,,,,
LOK,0.000000,28.576142,29.268622,27.070272,25.880838,32.049004,32.338314,27.345969,26.987305,27.060062,20.127414,29.535628
ABL1,28.576142,0.000000,20.448719,24.263294,24.275118,22.898073,23.211865,17.856452,17.480245,8.191927,20.824617,20.386950
DDR1,29.268622,20.448719,0.000000,24.935382,26.078996,23.450612,23.192182,26.258210,27.292591,17.887989,23.689144,7.628437
MYT1,27.070272,24.263294,24.935382,0.000000,6.680548,25.098772,25.161640,25.959846,29.273259,23.767301,23.136211,24.507839
MYT1,25.880838,24.275118,26.078996,6.680548,0.000000,25.598288,25.751657,26.249652,28.653591,23.771006,23.395898,25.658868
EphA2,32.049004,22.898073,23.450612,25.098772,25.598288,0.000000,3.898270,27.826531,29.753711,19.909221,21.697771,22.961274
EphA2,32.338314,23.211865,23.192182,25.161640,25.751657,3.898270,0.000000,27.827668,29.961407,20.418159,22.328941,22.773133
ABL1,27.345969,17.856452,26.258210,25.959846,26.249652,27.826531,27.827668,0.000000,18.140601,16.351846,20.976468,25.701912
ABL1,26.987305,17.480245,27.292591,29.273259,28.653591,29.753711,29.961407,18.140601,0.000000,12.611501,22.060313,27.975786


In [19]:
# # We unstack the matrix (each pairwise comparison in a single row)
# We group by kinase names (level=[0, 1] ensures that the order of the kinases is ignored
# We take the minimum value in each kinase pair group
# We unstack the remaining data points
kinase_distance_matrix_df = (
    kinase_distance_matrix_df.unstack().groupby(level=[0, 1]).min().unstack(level=1)
)
# Cosmetics: Remove the index and column names
kinase_distance_matrix_df.index.name = None
kinase_distance_matrix_df.columns.name = None

print(
    f"Structure matrix of shape {structure_distance_matrix_df.shape} "
    f"reduced to kinase matrix of shape {kinase_distance_matrix_df.shape}."
)
# NBVAL_CHECK_OUTPUT

Structure matrix of shape (12, 12) reduced to kinase matrix of shape (5, 5).


In [20]:
# plot the distance matrix. lighter colors indicate similarity, darker colors dissimilarity.
import seaborn as sns

# Show matrix with background gradient
cm = sns.light_palette("green", as_cmap=True)
kinase_distance_matrix_df.style.background_gradient(cmap=cm).format("{:.3f}")

,ABL1,DDR1,EphA2,LOK,MYT1
ABL1,0.000,17.888,19.909,20.825,23.767
DDR1,17.888,0.000,22.773,23.689,24.508
EphA2,19.909,22.773,0.000,21.698,25.099
LOK,20.825,23.689,21.698,0.000,23.136
MYT1,23.767,24.508,25.099,23.136,0.000


# Calculate ligand-aware KiSSim fingerprints

1. fetch ligand coordinates
2. fetch pocket coordinates
3. calculate distances:
    - distances_ctd : Distance from pocket residues to ligand centroid.
    - distances_cst : Distance from pocket residues to ligand closest heavy atom from the residue.
    - distances_fst : Distance from pocket residues to ligand furthest heavy atom from the residue.
    - distances_ftf : Distance from pocket residues to the ligand atom farthest from fct.
4. add ligand distance matrix to kissim

In [46]:
df_dasatinib_bosutinib.columns

Index(['structure.klifs_id', 'structure.pdb_id', 'structure.alternate_model',
       'structure.chain', 'species.klifs', 'kinase.klifs_id',
       'kinase.klifs_name', 'kinase.names', 'kinase.family', 'kinase.group',
       'structure.pocket', 'ligand.expo_id', 'ligand_allosteric.expo_id',
       'ligand.klifs_id', 'ligand_allosteric.klifs_id', 'ligand.name',
       'ligand_allosteric.name', 'structure.dfg', 'structure.ac_helix',
       'structure.resolution', 'structure.qualityscore',
       'structure.missing_residues', 'structure.missing_atoms',
       'structure.rmsd1', 'structure.rmsd2', 'interaction.fingerprint',
       'structure.front', 'structure.gate', 'structure.back', 'structure.fp_i',
       'structure.fp_ii', 'structure.bp_i_a', 'structure.bp_i_b',
       'structure.bp_ii_in', 'structure.bp_ii_a_in', 'structure.bp_ii_b_in',
       'structure.bp_ii_out', 'structure.bp_ii_b', 'structure.bp_iii',
       'structure.bp_iv', 'structure.bp_v', 'structure.grich_distance',
       

In [47]:
df_dasatinib_bosutinib[['structure.klifs_id', 'structure.pdb_id', 'structure.chain', 'kinase.klifs_name', 'species.klifs', 'structure.pocket', 'structure.dfg', 'structure.resolution', 'structure.qualityscore', 'ligand.expo_id']]

,structure.klifs_id,structure.pdb_id,structure.chain,kinase.klifs_name,species.klifs,structure.pocket,structure.dfg,structure.resolution,structure.qualityscore,ligand.expo_id
26,955,5ajq,A,LOK,Human,GELGDGAFGKVYKAAAKVIDYIVEIEILATCDPYIVKLLGAWIMIE...,in,2.20,9.6,DB8
147,1057,3ue4,A,ABL1,Human,HKLGGGQYGEVYEVAVKTLEFLKEAAVMKEIKPNLVQLLGVYIITE...,out-like,2.42,8.9,DB8
266,10761,6bsd,A,DDR1,Human,EKLGEGQFGEVHLVAVKILDFLKEVKIMSRLKPNIIRLLGVCMITD...,out-like,2.61,8.4,1N1
11,8339,5vcv,A,MYT1,Human,SRLGHGSYGEVFKYAVKRSRKLAEVGSHEKVGPCCVRLEQAYLQTE...,in,1.92,8.0,1N1
17,8345,5vcy,A,MYT1,Human,SRLGHGSYGEVFKYAVKRSRKLAEVGSHEKVGPCCVRLEQAYLQTE...,in,1.56,8.0,DB8
401,7177,5i9x,A,EphA2,Human,KVIGAGEFGEVYKVAIKTLDFLGEAGIMGQFSHNIIRLEGVMIITE...,in,1.43,8.0,DB8
392,7176,5i9y,A,EphA2,Human,KVIGAGEFGEVYKVAIKTLDFLGEAGIMGQFSHNIIRLEGVMIITE...,in,1.23,8.0,1N1
219,1053,2gqg,A,ABL1,Human,HKLGGGQYGEVYEVAVKTLEFLKEAAVMKEIKPNLVQLLGVYIITE...,in,2.40,8.0,1N1
103,14197,7n9g,A,ABL1,Human,HKLGGGQYGEVYEVAVKTLEFLKEAAVMKEIKPNLVQLLGVYIITE...,out-like,2.20,8.0,1N1
92,1094,4xey,A,ABL1,Human,HKLGG____EVYEVAVKTLEFLKEAAVMKEIKPNLVQLLGVYIITE...,out-like,2.89,6.4,1N1


## Fetch the PDB file using PandasPdb

In [ ]:
from biopandas.pdb import PandasPdb

pdb_id = '5ajq'
klifs_id = 955

ppdb = PandasPdb().fetch_pdb(pdb_id)
df_prot = ppdb.df['ATOM']
df_prot

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,1,,N,,GLU,,A,24,,...,-2.540,-8.335,39.576,1.0,91.52,,,N,NaN,587
1,ATOM,2,,CA,,GLU,,A,24,,...,-3.251,-7.031,39.756,1.0,89.08,,,C,NaN,588
2,ATOM,3,,C,,GLU,,A,24,,...,-3.416,-6.568,41.224,1.0,88.80,,,C,NaN,589
3,ATOM,4,,O,,GLU,,A,24,,...,-4.321,-5.775,41.540,1.0,81.37,,,O,NaN,590
4,ATOM,5,,CB,,GLU,,A,24,,...,-4.621,-7.079,39.057,1.0,89.94,,,C,NaN,591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4181,ATOM,4183,,N,,ALA,,B,312,,...,32.539,14.555,6.516,1.0,129.42,,,N,NaN,4769
4182,ATOM,4184,,CA,,ALA,,B,312,,...,33.862,13.917,6.554,1.0,126.18,,,C,NaN,4770
4183,ATOM,4185,,C,,ALA,,B,312,,...,34.590,14.119,5.226,1.0,127.98,,,C,NaN,4771
4184,ATOM,4186,,O,,ALA,,B,312,,...,35.790,14.392,5.194,1.0,131.43,,,O,NaN,4772


## get pocket residues

In [ ]:
klifs_session = setup_remote()
residues = klifs_session.pockets.by_structure_klifs_id(klifs_id)
pocket_residues_ids = residues['residue.id'].astype('int').to_list()
pocket_residues_ids

[40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 62,
 63,
 64,
 65,
 66,
 67,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 173,
 174,
 175,
 176,
 177,
 178,
 179]

## calculate the centoids of the pocket residues


In [ ]:
pocket_residues = df_prot[df_prot['residue_number'].isin(pocket_residues_ids)]
pocket_residues


,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
133,ATOM,134,,N,,GLY,,A,40,,...,21.495,-4.505,39.434,1.0,65.73,,,N,NaN,720
134,ATOM,135,,CA,,GLY,,A,40,,...,22.038,-3.137,39.441,1.0,67.67,,,C,NaN,721
135,ATOM,136,,C,,GLY,,A,40,,...,21.077,-2.083,39.941,1.0,64.54,,,C,NaN,722
136,ATOM,137,,O,,GLY,,A,40,,...,19.962,-2.383,40.376,1.0,64.05,,,O,NaN,723
137,ATOM,138,,N,,GLU,,A,41,,...,21.549,-0.847,39.897,1.0,68.89,,,N,NaN,724
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3239,ATOM,3241,,CA,,SER,,B,179,,...,7.553,33.070,-8.067,1.0,139.14,,,C,NaN,3827
3240,ATOM,3242,,C,,SER,,B,179,,...,7.249,34.470,-7.527,1.0,142.57,,,C,NaN,3828
3241,ATOM,3243,,O,,SER,,B,179,,...,6.108,34.932,-7.624,1.0,141.17,,,O,NaN,3829
3242,ATOM,3244,,CB,,SER,,B,179,,...,7.147,32.023,-7.007,1.0,134.76,,,C,NaN,3830


In [64]:
grouped = pocket_residues.groupby(['chain_id', 'residue_number', 'residue_name'])
centroids = grouped[['x_coord', 'y_coord', 'z_coord']].mean().reset_index()
centroids

,chain_id,residue_number,residue_name,x_coord,y_coord,z_coord
0,A,40,GLY,21.143000,-3.027000,39.798000
1,A,41,GLU,21.848222,0.626000,41.626333
2,A,42,LEU,18.390125,2.452750,37.868750
3,A,43,GLY,20.409500,6.223750,39.269500
4,A,44,ASP,20.959375,8.788375,41.808375
...,...,...,...,...,...,...
164,B,175,ASP,14.771375,30.466250,-4.077875
165,B,176,PHE,10.076818,29.419636,-3.197636
166,B,177,GLY,12.305500,32.985250,-7.133500
167,B,178,VAL,10.567286,32.940857,-10.465571


## get ligand pdb

In [67]:
hetatm = ppdb.df['HETATM']
ligand = hetatm[hetatm['residue_name'].isin(['DB8', '1N1'])]
ligand

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,HETATM,4189,,CAA,,DB8,,A,800,,...,16.543,6.047,36.608,1.0,47.02,,,C,NaN,4775
1,HETATM,4190,,O02,,DB8,,A,800,,...,9.487,1.975,40.790,1.0,45.40,,,O,NaN,4776
2,HETATM,4191,,CAC,,DB8,,A,800,,...,19.204,3.895,31.888,1.0,61.94,,,C,NaN,4777
3,HETATM,4192,,NAD,,DB8,,A,800,,...,8.511,2.972,36.880,1.0,38.33,,,N,NaN,4778
4,HETATM,4193,,CL1,,DB8,,A,800,,...,7.865,4.098,41.839,1.0,53.47,,,CL,NaN,4779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,HETATM,4261,,CBF,,DB8,,B,800,,...,22.040,27.608,3.283,1.0,95.46,,,C,NaN,4847
73,HETATM,4262,,CBG,,DB8,,B,800,,...,22.012,28.384,2.093,1.0,95.18,,,C,NaN,4848
74,HETATM,4263,,NBH,,DB8,,B,800,,...,28.793,23.780,3.126,1.0,131.86,,,N,NaN,4849
75,HETATM,4264,,NBI,,DB8,,B,800,,...,30.813,21.714,3.183,1.0,141.28,,,N,NaN,4850


## Calculate the distances
- distance_ctd : Distance from pocket residues to ligand centroid.
- distance_cst : Distance from pocket residues to ligand closest heavy atom from the residue.
- distance_fst : Distance from pocket residues to ligand furthest heavy atom from the residue.
- distance_ftf : Distance from pocket residues to the ligand atom farthest from fct.

In [68]:
ligand_centroid = ligand[['x_coord', 'y_coord', 'z_coord']].mean().values
ligand_centroid

array([18.61691667, 15.95843056, 18.57615278])

In [75]:
import pandas as pd
import numpy as np

def compute_distances(residues_df, ligand_df):
    residue_coords = residues_df[['x_coord', 'y_coord', 'z_coord']].to_numpy()
    ligand_coords = ligand_df[['x_coord', 'y_coord', 'z_coord']].to_numpy()
    ligand_centroid = ligand_coords.mean(axis=0)

    centroid_distances = []
    closest_atom_distances = []
    farthest_atom_distances = []
    ftf_distances = []

    for res_coord in residue_coords:
        # Distance to centroid
        d_centroid = np.linalg.norm(res_coord - ligand_centroid)

        # Distances from this residue to all ligand atoms
        dists = np.linalg.norm(ligand_coords - res_coord, axis=1)

        # Closest and farthest atom indices
        idx_closest = np.argmin(dists)
        idx_farthest = np.argmax(dists)
        fst_coord = ligand_coords[idx_farthest]

        # Now compute distances from fst to all other ligand atoms
        fst_to_others = np.linalg.norm(ligand_coords - fst_coord, axis=1)
        idx_ftf = np.argmax(fst_to_others)
        ftf_coord = ligand_coords[idx_ftf]

        # Distance from residue to ftf
        d_ftf = np.linalg.norm(res_coord - ftf_coord)

        # Append all distances
        centroid_distances.append(d_centroid)
        closest_atom_distances.append(dists[idx_closest])
        farthest_atom_distances.append(dists[idx_farthest])
        ftf_distances.append(d_ftf)

    # Add results to DataFrame
    residues_df = residues_df.copy()
    residues_df['dist_ctd'] = centroid_distances
    residues_df['dist_cst'] = closest_atom_distances
    residues_df['dist_fst'] = farthest_atom_distances
    residues_df['dist_ftf'] = ftf_distances

    return residues_df

# Apply the function to the pocket residues
compute_distances(centroids, ligand)


,chain_id,residue_number,residue_name,x_coord,y_coord,z_coord,dist_ctd,dist_cst,dist_fst,dist_ftf
0,A,40,GLY,21.143000,-3.027000,39.798000,28.586613,10.210525,55.088614,11.734462
1,A,41,GLU,21.848222,0.626000,41.626333,27.871770,9.094850,54.225130,11.882219
2,A,42,LEU,18.390125,2.452750,37.868750,23.551203,4.106674,49.910800,8.830868
3,A,43,GLY,20.409500,6.223750,39.269500,22.938875,4.697302,48.921252,11.746827
4,A,44,ASP,20.959375,8.788375,41.808375,24.426071,7.352782,49.753120,13.630242
...,...,...,...,...,...,...,...,...,...,...
164,B,175,ASP,14.771375,30.466250,-4.077875,27.174804,4.238295,53.548243,4.660219
165,B,176,PHE,10.076818,29.419636,-3.197636,26.985834,7.922333,52.027119,7.922333
166,B,177,GLY,12.305500,32.985250,-7.133500,31.475908,7.370165,57.326049,7.370165
167,B,178,VAL,10.567286,32.940857,-10.465571,34.592212,11.104113,60.044548,11.104113


# Calculate ligand-aware KiSSim fingerprint (easy pipeline)

In [214]:
import pandas as pd
import numpy as np

def compute_distances(residues_df, ligand_df):
    residue_coords = residues_df[['x_coord', 'y_coord', 'z_coord']].to_numpy()
    ligand_coords = ligand_df[['x_coord', 'y_coord', 'z_coord']].to_numpy()
    ligand_centroid = ligand_coords.mean(axis=0)

    centroid_distances = []
    closest_atom_distances = []
    farthest_atom_distances = []
    ftf_distances = []

    for res_coord in residue_coords:
        if pd.isnull(res_coord).all():
            centroid_distances.append(np.nan) # ctd
            closest_atom_distances.append(np.nan) # cst
            farthest_atom_distances.append(np.nan)
            ftf_distances.append(np.nan)

        else:
            # Distance to centroid (ctd)
            d_centroid = np.linalg.norm(res_coord - ligand_centroid)

            # Distances from this residue to all ligand atoms
            dists = np.linalg.norm(ligand_coords - res_coord, axis=1)

            # Closest and farthest atom indices
            idx_closest = np.argmin(dists) # cst
            idx_farthest = np.argmax(dists) # fst
            fst_coord = ligand_coords[idx_farthest]

            # Now compute distances from fst to all other ligand atoms
            fst_to_others = np.linalg.norm(ligand_coords - fst_coord, axis=1)
            idx_ftf = np.argmax(fst_to_others)
            ftf_coord = ligand_coords[idx_ftf]

            # Distance from residue to ftf
            d_ftf = np.linalg.norm(res_coord - ftf_coord)

            # Append all distances
            centroid_distances.append(d_centroid) # ctd
            closest_atom_distances.append(dists[idx_closest]) # cst
            farthest_atom_distances.append(dists[idx_farthest])
            ftf_distances.append(d_ftf)

    # Add results to DataFrame
    residues_df = pd.DataFrame()
    residues_df['dist_ctd'] = centroid_distances
    residues_df['dist_cst'] = closest_atom_distances
    residues_df['dist_fst'] = farthest_atom_distances
    residues_df['dist_ftf'] = ftf_distances

    return residues_df

In [ ]:
def ligand_feature_generator(pdb_id, klifs_id, chain_id):
    """
    Generate ligand features for a given PDB ID and KLIFS ID.
    This function fetches the PDB structure, extracts the pocket residues,
    and computes distances to the ligand atoms.
    """
    # Fetch the PDB structure
    ppdb = PandasPdb().fetch_pdb(pdb_id)
    df_prot = ppdb.df['ATOM']
    # Check if the PDB ID is valid
    if df_prot.empty:
        print(f"Error: PDB ID {pdb_id} not found.")
        return None

    # Extract ligand information
    hetatm = ppdb.df['HETATM']
    df_ligand = hetatm[hetatm['residue_name'].isin(['DB8', '1N1'])]

    # Get pocket residues from KLIFS
    klifs_session = setup_remote()
    residues = klifs_session.pockets.by_structure_klifs_id(klifs_id)
    try:
        pocket_residues_ids = residues['residue.id'].astype('int').to_list()
    except ValueError as e:
        print(f"Found missing residues in pocket. {e}") # TODO: handle this case
        return None
   

    # Filter pocket residues from the PDB structure
    df_pocket_residues = df_prot[df_prot['residue_number'].isin(pocket_residues_ids)]
    df_pocket_residues = pocket_residues[pocket_residues['chain_id'] == chain_id]

    # Group pocket residues to get centroids
    grouped = df_pocket_residues.groupby(['chain_id', 'residue_number', 'residue_name'])
    df_residue_centroids = grouped[['x_coord', 'y_coord', 'z_coord']].mean().reset_index()

    if len(df_residue_centroids) != 85:
        print(f"Warning: Expected 85 residues, but found {len(df_residue_centroids)} residues.")

    # Compute distances
    distances = compute_distances(df_residue_centroids, df_ligand)

    # flatten the distances matrix into 1D
    distances_feature = distances.values.flatten()
    print(f"Distances feature shape: {distances_feature.shape}")
    return distances_feature


In [137]:
ligand_feature_generator('5ajq', 955, 'A')

Distances feature shape: (340,)


array([28.58661347, 10.21052467, 55.0886144 , 11.73446177, 27.87177041,
        9.09484982, 54.22513029, 11.88221892, 23.55120271,  4.10667352,
       49.91080023,  8.83086844, 22.93887486,  4.69730189, 48.92125196,
       11.746827  , 24.42607126,  7.35278214, 49.75311961, 13.63024244,
       25.64548412,  8.34190779, 50.42410841, 13.03350043, 27.75766201,
       10.35475613, 52.11555073, 13.3147983 , 27.17884819,  8.40045481,
       51.12531095, 11.84392847, 29.11211473,  8.73254183, 54.25999605,
        9.77005841, 30.04197651,  9.73598664, 55.84187016, 10.10598242,
       26.77273649,  5.66416564, 52.76680107,  6.05872331, 30.51102129,
        8.17138682, 56.798872  ,  8.17138682, 26.11982554,  7.21715882,
       52.50238954, 10.00294009, 27.46523072,  6.83084629, 53.22905375,
        7.18358532, 26.75259601,  3.80605082, 52.51425858,  3.80605082,
       29.63574141,  3.95986514, 55.54755669,  3.95986514, 30.51868386,
        4.02624466, 55.90110318,  4.43139559, 34.96817101,  8.56

# study case
Kd in bioactivity dataset Karaman-Davis
| Ligand (Name)       | LOK (klifs_id, PDB)        | ABL1 (klifs_id, PDB)        |
|---------------------|----------------------------|-----------------------------|
| 1N1 (Dasatinib)     | 1200 (8388, 5owr)          | 0.046 (1053, 2gqg)          |
| DB8 (Bosutinib)     | 7 (955, 5ajq)              | 0.057 (1057, 3ue4)          |



In [148]:
df_study_case = df_dasatinib_bosutinib[['structure.klifs_id', 'structure.pdb_id', 'structure.chain', 'kinase.klifs_name', 'species.klifs', 'structure.dfg', 'structure.resolution', 'structure.qualityscore', 'ligand.expo_id']]
df_study_case = df_study_case[df_study_case['structure.klifs_id'].isin([955, 8388, 1057, 1053])]

In [151]:
# compute distance for all structures
df_study_case['ligand_features'] = df_study_case.apply(
    lambda row: ligand_feature_generator(row['structure.pdb_id'], row['structure.klifs_id'], row['structure.chain']),
    axis=1
)
df_study_case

Distances feature shape: (340,)
Distances feature shape: (340,)
Distances feature shape: (340,)
Found missing residues in pocket. invalid literal for int() with base 10: '_'


,structure.klifs_id,structure.pdb_id,structure.chain,kinase.klifs_name,species.klifs,structure.dfg,structure.resolution,structure.qualityscore,ligand.expo_id,ligand_features
26,955,5ajq,A,LOK,Human,in,2.20,9.6,DB8,"[28.586613473969038, 10.210524668203883, 55.08..."
147,1057,3ue4,A,ABL1,Human,out-like,2.42,8.9,DB8,"[91.5880868521137, 88.47063654682269, 101.7898..."
219,1053,2gqg,A,ABL1,Human,in,2.40,8.0,1N1,"[59.593917009896366, 32.838654387779044, 92.87..."
56,8388,5owr,A,LOK,Human,in,2.30,6.4,1N1,None


## manually add the ligand features for 5owr (8388)


In [241]:

pdb_id = '5owr'
klifs_id = 8388
chain_id = 'A'

ppdb = PandasPdb().fetch_pdb(pdb_id)
df_prot = ppdb.df['ATOM']
df_prot = df_prot[df_prot['chain_id'] == 'A']

# Check if the PDB ID is valid
if df_prot.empty:
    print(f"Error: PDB ID {pdb_id} not found.")
# Extract ligand information
hetatm = ppdb.df['HETATM']
df_ligand = hetatm[hetatm['residue_name'].isin(['DB8', '1N1'])]

# Get pocket residues from KLIFS
klifs_session = setup_remote()
residues = klifs_session.pockets.by_structure_klifs_id(klifs_id)
pocket_residues_ids = residues['residue.id'].to_list()
pocket_residues_ids = [int(i) for i in pocket_residues_ids if i != '_']
missing_residues_id = [44, 45, 46, 47]
pocket_residues_ids.extend(missing_residues_id)
pocket_residues_ids

[40,
 41,
 42,
 43,
 48,
 49,
 50,
 51,
 52,
 62,
 63,
 64,
 65,
 66,
 67,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 44,
 45,
 46,
 47]

In [237]:
df_prot

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,1,,N,,TYR,,A,23,,...,-29.105,50.753,24.414,1.0,97.69,,,N,NaN,474
1,ATOM,2,,CA,,TYR,,A,23,,...,-27.843,51.070,23.673,1.0,98.62,,,C,NaN,475
2,ATOM,3,,C,,TYR,,A,23,,...,-27.457,49.917,22.741,1.0,100.93,,,C,NaN,476
3,ATOM,4,,O,,TYR,,A,23,,...,-27.253,48.794,23.206,1.0,104.50,,,O,NaN,477
4,ATOM,5,,CB,,TYR,,A,23,,...,-26.698,51.355,24.650,1.0,94.36,,,C,NaN,478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2153,ATOM,2154,,CA,,GLU,,A,317,,...,16.171,32.247,26.190,1.0,64.84,,,C,NaN,2627
2154,ATOM,2155,,C,,GLU,,A,317,,...,17.595,32.449,26.714,1.0,67.19,,,C,NaN,2628
2155,ATOM,2156,,O,,GLU,,A,317,,...,18.028,31.839,27.698,1.0,63.57,,,O,NaN,2629
2156,ATOM,2157,,CB,,GLU,,A,317,,...,15.513,33.608,25.926,1.0,64.70,,,C,NaN,2630


In [243]:
# add fake residues to the list
pdb_with_fake = df_prot.copy()
for residue_number in missing_residues_id:
    if residue_number not in pdb_with_fake['residue_number'].values:
        fake_residues = pd.DataFrame([
            {
                'residue_number': residue_number,        # Choose an ID not in pdb_df
                'residue_name': 'FAK',    # Fake residue name
                'chain_id': 'A',          # Can be any unused chain
                'x_coord': np.nan,
                'y_coord': np.nan,
                'z_coord': np.nan,
                'atom_name': 'X',        # You can add more atoms if needed
                # Add other columns as needed
            }
            # You can add more dicts here if you want multiple fake residues
        ])

        # Ensure column match
        fake_residues = fake_residues.reindex(columns=df_prot.columns)

        # Append to the PDB dataframe
        pdb_with_fake = pd.concat([pdb_with_fake, fake_residues], ignore_index=True)
pdb_with_fake.sort_values(by='residue_number', inplace=True)
pdb_with_fake[pdb_with_fake['residue_number'].isin(missing_residues_id)]

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
2158,NaN,NaN,NaN,X,NaN,FAK,NaN,A,44,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2159,NaN,NaN,NaN,X,NaN,FAK,NaN,A,45,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2160,NaN,NaN,NaN,X,NaN,FAK,NaN,A,46,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2161,NaN,NaN,NaN,X,NaN,FAK,NaN,A,47,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [244]:
# Filter pocket residues from the PDB structure
df_pocket_residues = pdb_with_fake[pdb_with_fake['residue_number'].isin(pocket_residues_ids)]
df_pocket_residues

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
130,ATOM,131.0,,N,,GLY,,A,40,,...,0.377,48.035,22.217,1.0,92.92,,,N,NaN,604.0
131,ATOM,132.0,,CA,,GLY,,A,40,,...,1.227,46.992,21.619,1.0,92.90,,,C,NaN,605.0
132,ATOM,133.0,,C,,GLY,,A,40,,...,0.577,46.268,20.454,1.0,95.32,,,C,NaN,606.0
133,ATOM,134.0,,O,,GLY,,A,40,,...,-0.521,46.631,20.021,1.0,94.46,,,O,NaN,607.0
142,ATOM,143.0,,OE2,,GLU,,A,41,,...,1.997,47.724,15.510,1.0,99.41,,,O,NaN,616.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,ATOM,1138.0,,CB,,SER,,A,179,,...,-20.010,31.676,12.967,1.0,42.68,,,C,NaN,1611.0
1135,ATOM,1136.0,,C,,SER,,A,179,,...,-20.896,32.858,10.950,1.0,45.29,,,C,NaN,1609.0
1134,ATOM,1135.0,,CA,,SER,,A,179,,...,-20.044,31.677,11.430,1.0,43.38,,,C,NaN,1608.0
1133,ATOM,1134.0,,N,,SER,,A,179,,...,-18.679,31.704,10.883,1.0,42.56,,,N,NaN,1607.0


In [245]:
df_pocket_residues[df_pocket_residues['residue_number'] == 46]

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
2160,NaN,NaN,NaN,X,NaN,FAK,NaN,A,46,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [246]:
# Group pocket residues to get centroids
grouped = df_pocket_residues.groupby(['chain_id', 'residue_number', 'residue_name'])
df_residue_centroids = grouped[['x_coord', 'y_coord', 'z_coord']].mean().reset_index()
df_residue_centroids

,chain_id,residue_number,residue_name,x_coord,y_coord,z_coord
0,A,40,GLY,0.415000,46.981500,21.077750
1,A,41,GLU,1.451111,45.092111,17.605222
2,A,42,LEU,-0.514625,40.443000,19.153250
3,A,43,GLY,0.090250,39.415750,14.651000
4,A,44,FAK,NaN,NaN,NaN
...,...,...,...,...,...,...
80,A,175,ASP,-12.614250,33.303625,14.429000
81,A,176,PHE,-17.572182,33.119545,16.079727
82,A,177,GLY,-16.440250,34.587500,10.731750
83,A,178,VAL,-17.036857,31.292429,9.047714


In [247]:
# Compute distances
distances = compute_distances(df_residue_centroids, df_ligand)

# flatten the distances matrix into 1D
distances_feature = distances.values.flatten()
print(f"Distances feature shape: {distances_feature.shape}")

Distances feature shape: (340,)


In [248]:
distances_feature

array([11.59512077, 10.77718761, 15.95686007, 13.58070366, 10.88353701,
        9.66324757, 14.67158511, 12.67997114,  5.82464274,  4.59872936,
       11.5781957 ,  9.18046539,  8.06936117,  5.62393924, 11.98642645,
       11.21289314,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan, 11.60416495,  8.8050378 , 17.87803867,
        8.8050378 , 10.47974947,  8.61985127, 16.58600228,  9.05314357,
        7.35795327,  6.67033831, 13.64510118,  8.70853984,  8.5974694 ,
        7.42014311, 14.56433046, 10.3597322 ,  9.1427535 ,  8.65950572,
       14.38533812, 12.39551701,  8.93911316,  7.19550264, 15.74728865,
       11.60945459,  6.58972186,  4.28681768, 15.12886087,  8.09000317,
        8.87979816,  4.95633639, 16.90899484,  8.05986937, 10.19990048,
        4.72844958, 18.87671366,  5.59048329, 14.67572404,  9.78

In [250]:
df_study_case

,structure.klifs_id,structure.pdb_id,structure.chain,kinase.klifs_name,species.klifs,structure.dfg,structure.resolution,structure.qualityscore,ligand.expo_id,ligand_features
26,955,5ajq,A,LOK,Human,in,2.20,9.6,DB8,"[28.586613473969038, 10.210524668203883, 55.08..."
147,1057,3ue4,A,ABL1,Human,out-like,2.42,8.9,DB8,"[91.5880868521137, 88.47063654682269, 101.7898..."
219,1053,2gqg,A,ABL1,Human,in,2.40,8.0,1N1,"[59.593917009896366, 32.838654387779044, 92.87..."
56,8388,5owr,A,LOK,Human,in,2.30,6.4,1N1,None


add the array above to `df_study_case`

In [252]:
df_study_case.at[56, 'ligand_features'] = distances_feature
df_study_case

,structure.klifs_id,structure.pdb_id,structure.chain,kinase.klifs_name,species.klifs,structure.dfg,structure.resolution,structure.qualityscore,ligand.expo_id,ligand_features
26,955,5ajq,A,LOK,Human,in,2.20,9.6,DB8,"[28.586613473969038, 10.210524668203883, 55.08..."
147,1057,3ue4,A,ABL1,Human,out-like,2.42,8.9,DB8,"[91.5880868521137, 88.47063654682269, 101.7898..."
219,1053,2gqg,A,ABL1,Human,in,2.40,8.0,1N1,"[59.593917009896366, 32.838654387779044, 92.87..."
56,8388,5owr,A,LOK,Human,in,2.30,6.4,1N1,"[11.595120769592869, 10.777187611455041, 15.95..."


## append ligand_feature to other KiSSim features

In [270]:
# turn ligand_features into a numpy array
import numpy as np

ligand_features = np.array(df_study_case['ligand_features'].tolist())
ligand_features_df = pd.DataFrame(
    ligand_features, 
    index=df_study_case['structure.klifs_id'].tolist(),
    columns=range(1032, 1032+ligand_features.shape[1])
    )
ligand_features_df

,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,...,1362,1363,1364,1365,1366,1367,1368,1369,1370,1371
955,28.586613,10.210525,55.088614,11.734462,27.871770,9.094850,54.225130,11.882219,23.551203,4.106674,...,52.711141,11.445743,31.184492,4.658244,54.777654,8.726106,29.106850,5.594731,52.951773,9.264801
1057,91.588087,88.470637,101.789819,94.686784,92.281703,89.632814,103.410524,94.574101,87.198250,84.727210,...,103.145765,85.365461,89.450475,84.345009,104.302339,88.180686,89.523204,84.177358,104.469997,88.272018
1053,59.593917,32.838654,92.873447,32.838654,55.929538,30.053088,89.130643,30.053088,55.872814,31.854404,...,89.085731,43.338146,59.953293,39.205646,90.930936,42.646655,55.928574,35.021656,87.164532,38.672606
8388,11.595121,10.777188,15.956860,13.580704,10.883537,9.663248,14.671585,12.679971,5.824643,4.598729,...,24.829078,7.913201,17.723360,10.646752,26.577618,11.140151,18.726789,11.620560,28.048174,11.885413


In [268]:
kissim_fingerprints_df_subset = kissim_fingerprints_df[kissim_fingerprints_df.index.isin(df_study_case['structure.klifs_id'].tolist())]
kissim_fingerprints_df_subset

,0,1,2,3,4,5,6,7,8,9,...,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031
structure.klifs_id,,,,,,,,,,,,,,,,,,,,,
955,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,2.0,0.0,...,12.935410,11.943754,4.392296,4.712463,4.455708,3.494710,2.748480,2.987596,3.679778,1.538272
1057,2.0,1.0,1.0,0.0,1.0,0.0,3.0,3.0,2.0,1.0,...,13.025692,12.011242,4.460881,5.034019,4.077518,3.423783,2.929926,3.687617,3.166885,2.050392
1053,2.0,1.0,1.0,0.0,1.0,0.0,3.0,3.0,2.0,1.0,...,12.822959,11.962476,4.493169,4.836616,4.315794,3.468795,3.217910,3.302662,3.380991,2.067249
8388,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,2.0,0.0,...,13.439285,12.091894,4.518169,4.811222,4.276097,3.536792,2.606818,3.305760,3.092917,1.469194


In [271]:
features_merged = pd.concat([kissim_fingerprints_df_subset, ligand_features_df], axis=1)
features_merged

,0,1,2,3,4,5,6,7,8,9,...,1362,1363,1364,1365,1366,1367,1368,1369,1370,1371
955,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,2.0,0.0,...,52.711141,11.445743,31.184492,4.658244,54.777654,8.726106,29.106850,5.594731,52.951773,9.264801
1057,2.0,1.0,1.0,0.0,1.0,0.0,3.0,3.0,2.0,1.0,...,103.145765,85.365461,89.450475,84.345009,104.302339,88.180686,89.523204,84.177358,104.469997,88.272018
1053,2.0,1.0,1.0,0.0,1.0,0.0,3.0,3.0,2.0,1.0,...,89.085731,43.338146,59.953293,39.205646,90.930936,42.646655,55.928574,35.021656,87.164532,38.672606
8388,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,2.0,0.0,...,24.829078,7.913201,17.723360,10.646752,26.577618,11.140151,18.726789,11.620560,28.048174,11.885413


In [272]:
features_merged.values

array([[  1.        ,   0.        ,   0.        , ...,   5.59473076,
         52.95177307,   9.26480127],
       [  2.        ,   1.        ,   1.        , ...,  84.17735788,
        104.46999681,  88.2720185 ],
       [  2.        ,   1.        ,   1.        , ...,  35.02165558,
         87.16453226,  38.672606  ],
       [  1.        ,   0.        ,   0.        , ...,  11.62056021,
         28.04817353,  11.88541304]])

## compare structures

In [274]:
structure_distance_matrix_array = nan_euclidean_distances(features_merged.values)

# Create DataFrame with structure KLIFS IDs as index/columns
structure_klifs_ids = features_merged.index.to_list()
structure_distance_matrix_df = pd.DataFrame(
    structure_distance_matrix_array, index=structure_klifs_ids, columns=structure_klifs_ids
)
print(f"Structure distance matrix size: {structure_distance_matrix_df.shape}")
structure_distance_matrix_df

Structure distance matrix size: (4, 4)


,955,1057,1053,8388
955,0.000000,1164.645008,650.061323,305.791963
1057,1164.645008,0.000000,607.199014,1335.469066
1053,650.061323,607.199014,0.000000,895.744168
8388,305.791963,1335.469066,895.744168,0.000000
